# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Alopecia'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [5]:
trait_subdir = "TCGA_Thyroid_Cancer_(THCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_THCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'age_at_initial_pathologic_diagnosis',
 'bcr_followup_barcode']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'braf_gene_genotyping_outcome_lab_results_text', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathol

In [12]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [13]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [74, 65, 55, 57, 33],
 'days_to_birth': [-27348, -24102, -20361, -21008, -12295]}

In [16]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [17]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [18]:
gender_col = 'gender'

In [19]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [20]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [21]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Thyroid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-4C-A93U-01,1,74,0,-0.767592,-1.742926,-0.531035,-0.067972,0.265922,-0.13531,0.440006,...,-0.748878,-1.018517,0.205567,-1.28139,0.395523,1.735414,0.945273,-0.186695,4.024767,-0.273612
TCGA-BJ-A0YZ-01,1,65,1,1.766308,0.273374,-0.531035,0.790028,1.209322,0.01229,0.704606,...,0.125322,1.568883,0.626367,2.38101,0.275823,1.820014,2.140073,1.380405,1.257367,-0.174312
TCGA-BJ-A0Z0-01,1,55,1,0.986808,-2.165726,-0.050935,0.051428,0.979522,0.48229,0.090106,...,-0.748878,-0.747417,-0.931133,-1.28139,-0.222077,0.293414,0.725173,-0.254095,0.530667,-0.430812
TCGA-BJ-A0Z2-01,1,57,1,0.767208,-1.623626,-0.531035,0.182228,0.541622,0.65889,0.647206,...,-0.748878,-1.587117,-1.292433,-1.28139,-0.602177,0.134114,0.894473,-0.380395,-2.502833,0.360588
TCGA-BJ-A0Z3-01,1,33,0,-0.287292,-1.766626,-0.531035,0.059628,0.920522,0.21849,0.336106,...,-0.413778,1.111583,1.158867,-1.28139,0.003723,1.240514,0.924373,-0.143495,2.613367,-0.030012


In [22]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 572 samples.


In [23]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Thyroid Cancer', the least common label is '0' with 59 occurrences. This represents 10.31% of the dataset.
The distribution of the feature 'Thyroid Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 46.0
  75%: 58.0
Min: 15
Max: 89
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 157 occurrences. This represents 27.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [24]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [25]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Jinglin/Thyroid-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [13]:
dataset = 'GEO'
trait_subdir = "Alopecia"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE81071',
 'GSE148346',
 'GSE101620',
 'GSE65391',
 'GSE66664',
 'GSE80342',
 'GSE18876',
 'GSE68801']

Repeat the below steps for all the accession numbers

In [14]:
cohort = accession_num = "GSE66664"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Alopecia/GSE66664/GSE66664_family.soft.gz',
 '../../DATA/GEO/Alopecia/GSE66664/GSE66664_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [15]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Transcriptome analysis reveals differences in vasculature signalling between human dermal papilla cells from balding and non-balding scalp"
!Series_summary	"Transcriptome analysis of hTERT-immortalized balding (BAB) and non-balding (BAN) dermal papilla cells derived from frontal and occipital scalp of male patients with androgenetic alopecia Hamilton grade IV. Interrogation of transcriptome differences between BAB and BAN after dihydrotestosterone (DHT, active metabolite of androgen) treatment revealed significant enrichment of vasculature-related genes among down-regulated genes in BAB compared to BAN."
!Series_overall_design	"RNA obtained from BAB and BAN after treatment with 1nM or 10nM of DHT, 2-3 replicates for each condition"


In [16]:
clinical_data.head()

,!Sample_geo_accession,GSM1627302,GSM1627303,GSM1627304,GSM1627305,GSM1627306,GSM1627307,GSM1627308,GSM1627309,GSM1627310,...,GSM1627432,GSM1627433,GSM1627434,GSM1627435,GSM1627436,GSM1627437,GSM1627438,GSM1627439,GSM1627440,GSM1627441
0,!Sample_characteristics_ch1,cell line: BAB,cell line: BAB,cell line: BAB,cell line: BAB,cell line: BAB,cell line: BAB,cell line: BAB,cell line: BAB,cell line: BAB,...,cell line: BAN,cell line: BAN,cell line: BAN,cell line: BAN,cell line: BAN,cell line: BAN,cell line: BAN,cell line: BAN,cell line: BAN,cell line: BAN
1,!Sample_characteristics_ch1,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,...,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT,agent: DHT
2,!Sample_characteristics_ch1,dose: 10nM,dose: 10nM,dose: 10nM,dose: 10nM,dose: 10nM,dose: 10nM,dose: 10nM,dose: 10nM,dose: 10nM,...,dose: 1nM,dose: 1nM,dose: 1nM,dose: 1nM,dose: 1nM,dose: 1nM,dose: 1nM,dose: 1nM,dose: 1nM,dose: 1nM
3,!Sample_characteristics_ch1,time (treatment duration): 0h,time (treatment duration): 0h,time (treatment duration): 0h,time (treatment duration): 12h,time (treatment duration): 12h,time (treatment duration): 12h,time (treatment duration): 15min,time (treatment duration): 15min,time (treatment duration): 15min,...,time (treatment duration): 36h,time (treatment duration): 36h,time (treatment duration): 3h,time (treatment duration): 3h,time (treatment duration): 48h,time (treatment duration): 48h,time (treatment duration): 48h,time (treatment duration): 6h,time (treatment duration): 6h,time (treatment duration): 6h


In [17]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['cell line: BAB', 'cell line: BAN'],
 1: ['agent: DHT'],
 2: ['dose: 10nM', 'dose: 1nM'],
 3: ['time (treatment duration): 0h',
  'time (treatment duration): 12h',
  'time (treatment duration): 15min',
  'time (treatment duration): 16h',
  'time (treatment duration): 1h',
  'time (treatment duration): 20h',
  'time (treatment duration): 24h',
  'time (treatment duration): 30min',
  'time (treatment duration): 36h',
  'time (treatment duration): 3h',
  'time (treatment duration): 48h',
  'time (treatment duration): 6h']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [18]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
   
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Alopecia\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Alopecia\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer.

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [26]:
age_row = gender_row = None
convert_age = convert_gender = None

In [27]:
is_gene_availabe = True
# is_gene_availabe = False
trait_row = 0
trait_type = 'binary'

age_row = None
# age_row = 4

gender_row = None
# gender_row = 3

In [28]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [29]:
def convert_trait(value):
    """
    Converts the given value to a binary data type based on whether it indicates balding or not.
    Returns 1 for balding, 0 for non-balding, and None for unknown values.

    :param value: The value to be converted (expected either 'cell line: BAB' or 'cell line: BAN').
    :return: Binary representation of the trait (1 or 0) or None.
    """
    if value == 'cell line: BAB':
        return 1  # Balding
    elif value == 'cell line: BAN':
        return 0  # Non-balding
    else:
        return None  # Unknown value

In [30]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1627302,GSM1627303,GSM1627304,GSM1627305,GSM1627306,GSM1627307,GSM1627308,GSM1627309,GSM1627310,GSM1627311,...,GSM1627432,GSM1627433,GSM1627434,GSM1627435,GSM1627436,GSM1627437,GSM1627438,GSM1627439,GSM1627440,GSM1627441
Alopecia,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


### Genetic data preprocessing and final filtering

In [33]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1627302,GSM1627303,GSM1627304,GSM1627305,GSM1627306,GSM1627307,GSM1627308,GSM1627309,GSM1627310,GSM1627311,...,GSM1627432,GSM1627433,GSM1627434,GSM1627435,GSM1627436,GSM1627437,GSM1627438,GSM1627439,GSM1627440,GSM1627441
ID,,,,,,,,,,,,,,,,,,,,,
ILMN_1343291,14.824485,14.824485,14.788861,14.728368,14.709844,14.709844,14.709844,14.788861,14.766228,14.824485,...,14.788861,14.691452,14.766228,14.824485,14.766228,14.766228,14.709844,14.674859,14.709844,14.824485
ILMN_1343295,13.033633,13.117320,12.770128,12.941854,13.044267,12.802287,12.866651,13.023121,12.651381,12.914239,...,12.347862,12.823303,12.990397,12.415548,13.108542,12.232160,12.797953,13.004498,12.433489,12.804583
ILMN_1651199,5.783288,5.431169,5.266022,5.693007,5.593886,5.564805,5.742819,5.628687,5.555745,5.669864,...,5.896344,5.700576,5.537633,5.553991,5.678688,5.559484,5.721028,5.583759,5.753718,5.679202
ILMN_1651209,5.827432,5.660542,5.920475,6.057376,5.820518,5.890001,5.760850,5.557443,5.852394,5.491696,...,5.930372,5.919904,5.743911,5.640238,5.930781,6.001091,5.972834,5.808267,5.850702,5.702466
ILMN_1651210,5.675353,5.707642,5.654719,5.587755,5.800296,5.749819,5.566861,5.565154,5.675260,5.960115,...,5.718479,5.809575,5.522222,5.676399,5.789016,5.755087,5.889422,5.729742,5.582547,5.781765


In [34]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['ILMN_1343291',
 'ILMN_1343295',
 'ILMN_1651199',
 'ILMN_1651209',
 'ILMN_1651210',
 'ILMN_1651221',
 'ILMN_1651228',
 'ILMN_1651229',
 'ILMN_1651230',
 'ILMN_1651232',
 'ILMN_1651235',
 'ILMN_1651236',
 'ILMN_1651237',
 'ILMN_1651238',
 'ILMN_1651249',
 'ILMN_1651253',
 'ILMN_1651254',
 'ILMN_1651259',
 'ILMN_1651260',
 'ILMN_1651262']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [35]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209', 'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262']\n"

If not required, jump directly to the gene normalization step

In [36]:
requires_gene_mapping = True

In [38]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGAGGGCGCGTAGTTTGCA

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [39]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_id_key = key_name1
gene_symb_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

"\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_id_key = key_name1\ngene_symb_key = key_name2\n\nGene annotation dictionary:\n{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_

In [41]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [42]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

14 input query terms found dup hits:	[('ABCC13', 2), ('ABCC6P1', 2), ('ABCC6P2', 3), ('ADAM6', 3), ('AGAP11', 2), ('AK2P2', 2), ('ALOX12P
106 input query terms found no hit:	['1-Dec', '1-Mar', '10-Mar', '11-Mar', '2-Mar', '3-Mar', '4-Mar', '5-Mar', '6-Mar', '7-Mar', '7A5', 
7 input query terms found dup hits:	[('ATP6AP1L', 2), ('ATXN8OS', 2), ('BAGE2', 2), ('BMP2KL', 2), ('BRD7P2', 2), ('BRD7P3', 2), ('BRI3P
361 input query terms found no hit:	['ARMC4', 'ARMET', 'ARNTL', 'ARNTL2', 'ARP11', 'ARPM1', 'ARPM2', 'ARPP-21', 'ARS2', 'ARSE', 'ARVP612
2 input query terms found dup hits:	[('C2orf83', 2), ('C5orf60', 2)]
810 input query terms found no hit:	['C15orf34', 'C15orf37', 'C15orf38', 'C15orf41', 'C15orf42', 'C15orf43', 'C15orf44', 'C15orf49', 'C1
8 input query terms found dup hits:	[('CATSPER2P1', 2), ('CCDC144NL', 2), ('CCNYL2', 2), ('CCNYL3', 2), ('CCT6P1', 2), ('CDR1', 2), ('CE
137 input query terms found no hit:	['C9orf82', 'C9orf84', 'C9orf86', 'C9orf89', 'C9orf9', 'C9orf90', 'C9orf

In [43]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [44]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 140 samples.


In [45]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Alopecia', the least common label is '0.0' with 69 occurrences. This represents 49.29% of the dataset.
The distribution of the feature 'Alopecia' in this dataset is fine.



False

In [46]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [47]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [4]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
4,GSE66664,True,True,False,False,False,140.0,


In [6]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

In [7]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [8]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [198]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [200]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [201]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [202]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 83.16% ± 5.64%


In [203]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [204]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

if has_batch_effect:
    report_result_from_lmm(model, feature_cols, TRAIT, condition, threshold=0.05, save_output=True,
                           output_dir=OUTPUT_DIR)
else:
    report_result_from_lasso(model, feature_cols, TRAIT, condition, save_output=True, output_dir=OUTPUT_DIR)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0112
Found 100 significant genes associated with the trait 'Thyroid Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
     GABRB2     0.199853       9.445009e-24
       LRP4     0.153916       3.454747e-11
    SHROOM4     0.150237       3.783756e-09
      HMGA2     0.122833       4.533386e-08
       EPS8     0.148314       3.922968e-07
    KLHDC8A     0.103442       3.996388e-07
       LIPH     0.126974       5.203673e-07
      MPZL2     0.123835       7.878797e-07
      GABRD     0.085857       3.424810e-06
       GJB3     0.104297       7.705250e-06
   C6orf168     0.106365       1.189886e-05
        FN1     0.142549       1.760688e-05
      KCNQ3     0.132879       2.031323e-05
      GRHL3     0.103693       2.317901e-05
    ZCCHC12     0.091659       3.351467e-05
     ACTBL2     0.094812       6.476432e-05
      SYT12     0.127073       8.24

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
